In [1]:
import pandas as pd
import requests as req
import numpy as np
from tqdm.notebook import tqdm
import folium
from folium.plugins import MarkerCluster

from bs4 import BeautifulSoup as bs

from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
url = 'https://chargefinder.com/en/washington/charging-station/m76w89'

html = req.get(url).content 

soup = bs(html, 'html.parser')

In [41]:
soup.find_all('div')

[<div id="app"></div>]

In [2]:
df = pd.read_csv('/Users/nico/Desktop/ironhack/proyectos/Project_ETL/messy_data/Electric_Vehicle_Charging_Stations.csv')

In [3]:
df

,Station Name,Street Address,City,Access Days Time,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Other Info,New Georeferenced Column
0,BMW OF DARIEN,138-142 Ledge Rd,Darien,24 hours daily,NONE,2,NONE,NONE,POINT (-73.4764687 41.072882)
1,Dunkin’ - Tesla Supercharger,893 E Main St,Meriden,24 hours daily; for Tesla use only,NONE,NONE,8,NONE,POINT (-72.773473 41.527367)
2,Town of Beacon Falls - Commuter Lot,105 N Main St,Beacon Falls,24 hours daily,NONE,1,NONE,NONE,POINT (-73.065583 41.44548100000001)
3,OLD SAYBROOK VW,319 Middlesex Turnpike,Old Saybrook,24 hours daily,NONE,2,NONE,NONE,POINT (-72.3825 41.3102778)
4,Fairfield Rail Station,80 Mill Plain Rd,Fairfield,24 hours daily,NONE,2,NONE,NONE,POINT (-73.264511 41.143125)
...,...,...,...,...,...,...,...,...,...
380,Lee Partyka Chevrolet Mazda Isuzu,200 Skiff St,Hamden,Dealership business hours,NONE,3,NONE,NONE,POINT (-72.9100121 41.3729709)
381,777 MAIN ST,71 Asylum St,Hartford,24 hours daily,NONE,10,1,NONE,POINT (-72.674468 41.766676)
382,Norwich Public Utilities,173 N Main St,Norwich,24 hours daily,NONE,4,NONE,NONE,POINT (-72.063516 41.52757900000001)
383,Fairfield Plaza,116 Danbury Rd,New Milford,24 hours daily,NONE,NONE,1,NONE,POINT (-73.41763 41.55735)


In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Station Name              385 non-null    object
 1   Street Address            385 non-null    object
 2   City                      385 non-null    object
 3   Access Days Time          385 non-null    object
 4   EV Level1 EVSE Num        385 non-null    object
 5   EV Level2 EVSE Num        385 non-null    object
 6   EV DC Fast Count          385 non-null    object
 7   EV Other Info             385 non-null    object
 8   New Georeferenced Column  385 non-null    object
dtypes: object(9)
memory usage: 237.7 KB


In [5]:
df1 = df.replace('NONE',np.nan)

In [6]:
df1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Station Name              385 non-null    object
 1   Street Address            385 non-null    object
 2   City                      385 non-null    object
 3   Access Days Time          385 non-null    object
 4   EV Level1 EVSE Num        9 non-null      object
 5   EV Level2 EVSE Num        341 non-null    object
 6   EV DC Fast Count          54 non-null     object
 7   EV Other Info             2 non-null      object
 8   New Georeferenced Column  385 non-null    object
dtypes: object(9)
memory usage: 205.6 KB


In [7]:
df1.drop(columns= 'EV Level1 EVSE Num',inplace=True)
df1.drop(columns= 'EV DC Fast Count',inplace=True)
df1.drop(columns= 'EV Other Info',inplace=True)

In [8]:
df1 = df1.dropna()

In [9]:
df1.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 384
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Station Name              341 non-null    object
 1   Street Address            341 non-null    object
 2   City                      341 non-null    object
 3   Access Days Time          341 non-null    object
 4   EV Level2 EVSE Num        341 non-null    object
 5   New Georeferenced Column  341 non-null    object
dtypes: object(6)
memory usage: 152.1 KB


In [10]:
df1 = df1.drop_duplicates()

In [11]:
df1.shape

(341, 6)

In [12]:
df1.City.unique()

array(['Darien', 'Beacon Falls', 'Old Saybrook', 'Fairfield', 'Norwich',
       'Glastonbury', 'Groton', 'Waterbury', 'Farmington', 'Shelton',
       'Wallingford', 'Rocky Hill', 'New Haven', 'Easton', 'Colebrook',
       'Mystic', 'Windsor Locks', 'Storrs', 'Madison', 'New Britain',
       'Mansfield', 'Torrington', 'Harwinton', 'Middletown',
       'North Windham', 'Westbrook', 'Bristol', 'Branford', 'Manchester',
       'Meriden', 'Guilford', 'Simsbury', 'Hartford', 'Stratford',
       'Stonington', 'Westport', 'Stamford', 'Niantic', 'New Milford',
       'Newtown', 'Bloomfield', 'Washington', 'North Haven',
       'Stafford Springs', 'West Hartford', 'Newington', 'Norwalk',
       'Southbury', 'Ridgefield', 'Mashantucket', 'East Hartford',
       'Ledyard', 'Plainville', 'Greenwich', 'New London', 'Windham',
       'Danbury', 'Canton', 'Durham', 'Southington', 'New Fairfield',
       'Orange', 'West Haven', 'Bridgeport', 'Coventry', 'Uncasville',
       'Enfield', 'Ellington', 'Ber

In [13]:
df1[['Latitude', 'Longitude']] = df1['New Georeferenced Column'].str.strip('POINT ()').str.split(expand=True)

In [14]:
df1.drop(columns='New Georeferenced Column',inplace= True )

In [15]:
df1['Access Days Time'].unique()

array(['24 hours daily',
       '24 hours daily; guest use only; see front desk for access',
       '6am-12am daily', 'Dealership business hours',
       '24 hours daily; for customer use only; see front desk for access',
       '24 hours daily; pay lot', '24 hours daily; for customer use only',
       '8am-3pm M-F', '24 hours daily; permit lot', '6am-12pm daily',
       '7am-7pm M-F', 'Garage business hours',
       'MO: Not Specified; TU: Not Specified; WE: Not Specified; TH: Not Specified; FR: Not Specified; SA: Not Specified; SU: Not Specified',
       'Restricted to garage hours; pay lot',
       'MO: 12:00am-12:00am; TU: 12:00am-12:00am; WE: 12:00am-12:00am; TH: 12:00am-12:00am; FR: 12:00am-12:00am; SA: 12:00am-12:00am; SU: 12:00am-12:00am',
       'School hours', '6am-12am',
       '8am-10pm daily; for customer use only; see inside for access',
       '24 hours daily; for member use only; see valet for access',
       '24 hours daily; overnight valet fee',
       '24 hours daily

In [16]:
def clean(x):
    x=str(x)
    if '24' in x:
        return '24 Hours'
    else:
        return 'Regular'
df1['Access Days Time'] = df1['Access Days Time'].apply(clean)

# 

In [17]:
df1

,Station Name,Street Address,City,Access Days Time,EV Level2 EVSE Num,Latitude,Longitude
0,BMW OF DARIEN,138-142 Ledge Rd,Darien,24 Hours,2,-73.4764687,41.072882
2,Town of Beacon Falls - Commuter Lot,105 N Main St,Beacon Falls,24 Hours,1,-73.065583,41.44548100000001
3,OLD SAYBROOK VW,319 Middlesex Turnpike,Old Saybrook,24 Hours,2,-72.3825,41.3102778
4,Fairfield Rail Station,80 Mill Plain Rd,Fairfield,24 Hours,2,-73.264511,41.143125
5,FOUNDRY66,50 Franklin St,Norwich,24 Hours,2,-72.0741188,41.525611
...,...,...,...,...,...,...,...
379,Yankee Doodle,10 Burnell Boulevard,Norwalk,24 Hours,1,-73.414932,41.11803
380,Lee Partyka Chevrolet Mazda Isuzu,200 Skiff St,Hamden,Regular,3,-72.9100121,41.3729709
381,777 MAIN ST,71 Asylum St,Hartford,24 Hours,10,-72.674468,41.766676
382,Norwich Public Utilities,173 N Main St,Norwich,24 Hours,4,-72.063516,41.52757900000001


In [18]:
df1 = df1.rename(columns= {'EV Level2 EVSE Num': 'EVSE units'})

In [19]:
df1

,Station Name,Street Address,City,Access Days Time,EVSE units,Latitude,Longitude
0,BMW OF DARIEN,138-142 Ledge Rd,Darien,24 Hours,2,-73.4764687,41.072882
2,Town of Beacon Falls - Commuter Lot,105 N Main St,Beacon Falls,24 Hours,1,-73.065583,41.44548100000001
3,OLD SAYBROOK VW,319 Middlesex Turnpike,Old Saybrook,24 Hours,2,-72.3825,41.3102778
4,Fairfield Rail Station,80 Mill Plain Rd,Fairfield,24 Hours,2,-73.264511,41.143125
5,FOUNDRY66,50 Franklin St,Norwich,24 Hours,2,-72.0741188,41.525611
...,...,...,...,...,...,...,...
379,Yankee Doodle,10 Burnell Boulevard,Norwalk,24 Hours,1,-73.414932,41.11803
380,Lee Partyka Chevrolet Mazda Isuzu,200 Skiff St,Hamden,Regular,3,-72.9100121,41.3729709
381,777 MAIN ST,71 Asylum St,Hartford,24 Hours,10,-72.674468,41.766676
382,Norwich Public Utilities,173 N Main St,Norwich,24 Hours,4,-72.063516,41.52757900000001


In [53]:
df1.latitude

0     -73.476469
2     -73.065583
3     -72.382500
4     -73.264511
5     -72.074119
         ...    
379   -73.414932
380   -72.910012
381   -72.674468
382   -72.063516
384   -72.208134
Name: latitude, Length: 341, dtype: float64

In [35]:
m = folium.Map(location= [ 41.730405,-72.790618 ], tiles = 'OpenStreetMap', zoom_start = 10)

In [21]:
df1['EVSE units']= df1['EVSE units'].astype('int64')

In [29]:
df1['latitude']= df1['latitude'].astype('float64')

In [28]:
df1['longitude']= df1['longitude'].astype('float64')

In [37]:
for i,row in df1.iterrows():
    latitude = df1.at[i, 'latitude']
    longitude = df1.at[i, 'longitude']
    name = df1.at[i, 'station_name']
    
    folium.Marker(location= [longitude, latitude],popup= name,icon= folium.Icon(color= 'blue')).add_to(m)


In [26]:
df1.rename(columns=lambda col: col.lower().replace(" ", "_"), inplace=True)

In [51]:
df1.info(memory_usage= 'deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341 entries, 0 to 384
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   station_name      341 non-null    object 
 1   street_address    341 non-null    object 
 2   city              341 non-null    object 
 3   access_days_time  341 non-null    object 
 4   evse_units        341 non-null    int64  
 5   latitude          341 non-null    float64
 6   longitude         341 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 114.1 KB


In [67]:
df1.to_csv('../data/Clean_Charging_stations.csv')

In [38]:
m


In [50]:
map_html = m._repr_html_()
     
print({"map": map_html})

{'map': '<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><span style="color:#565656">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;html&gt;\n&lt;head&gt;\n    \n    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n    \n        &lt;script&gt;\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        &lt;/script&gt;\n    \n    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js&quot;&g

In [52]:
m.save("map.html")

In [35]:
lep = pd.read_csv('../data/Clean_Electric_Vehicle_Population_Data.csv')

In [36]:
lep.rename(columns=lambda col: col.lower().replace(" ", "_"), inplace=True)
lep.rename(columns= {'clean_alternative_fuel_vehicle_(cafv)_eligibility': 'cafv_eligibilaty'}, inplace = True)

In [37]:
lep.to_csv(('../data/Clean_Electric_Vehicle_Population_Data.csv'))

In [38]:
lep

,county,city,state,postal_code,model_year,make,model,electric_vehicle_type,cafv_eligibilaty,electric_range,vehicle_location,electric_utility,latitude,longitude
0,Yakima,Yakima,WA,98901,2011,NISSAN,LEAF,Electric,Clean Alternative Fuel Vehicle Eligible,73,(-120.50721 46.60448),PACIFICORP,-120.50721,46.60448
1,Skagit,Concrete,WA,98237,2017,CHEVROLET,BOLT EV,Electric,Clean Alternative Fuel Vehicle Eligible,238,(-121.7515 48.53892),PUGET SOUND ENERGY INC,-121.75150,48.53892
2,Snohomish,Everett,WA,98201,2019,FORD,FUSION,Plug-in Hybrid,Not eligible due to low battery range,26,(-122.20596 47.97659),PUGET SOUND ENERGY INC,-122.20596,47.97659
3,Snohomish,Bothell,WA,98021,2018,TESLA,MODEL 3,Electric,Clean Alternative Fuel Vehicle Eligible,215,(-122.18384 47.8031),PUGET SOUND ENERGY INC,-122.18384,47.80310
4,Snohomish,Everett,WA,98203,2013,NISSAN,LEAF,Electric,Clean Alternative Fuel Vehicle Eligible,75,(-122.23019 47.94949),PUGET SOUND ENERGY INC,-122.23019,47.94949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36277,Kitsap,Gig Harbor,WA,98329,2018,TESLA,MODEL 3,Electric,Clean Alternative Fuel Vehicle Eligible,215,(-122.72457 47.38165),PUGET SOUND ENERGY INC,-122.72457,47.38165
36278,King,Seattle,WA,98133,2019,SMART,EQ FORTWO,Electric,Clean Alternative Fuel Vehicle Eligible,58,(-122.3503 47.71868),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),-122.35030,47.71868
36279,Kitsap,Bainbridge Island,WA,98110,2018,MERCEDES-BENZ,C-CLASS,Plug-in Hybrid,Not eligible due to low battery range,8,(-122.521 47.62728),PUGET SOUND ENERGY INC,-122.52100,47.62728
36280,King,Seattle,WA,98177,2020,AUDI,Q5 E,Plug-in Hybrid,Not eligible due to low battery range,20,(-122.36498 47.72238),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),-122.36498,47.72238
